## Treinamento do modelo e exportação (treinar_modelo_45dias.py)



In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import save_model

# Fixar seed para reprodutibilidade
import random
import tensorflow as tf

def set_seeds(seed=1995):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)

set_seeds()

# Baixar dados
ticker = 'AAPL'
inicio = '2018-01-01'
fim = '2025-03-30'

df = yf.download(ticker, start=inicio, end=fim)
df.columns = df.columns.droplevel(0)
df.columns.name = None
df.columns = ['Preço_Fechamento', 'Máxima', 'Mínima', 'Preço_Abertura', 'Volume_Negociado']
df.reset_index(inplace=True)

# Criar Retorno_MM3
df['Retorno_Diário'] = df['Preço_Fechamento'].pct_change()
df['Retorno_MM3'] = df['Retorno_Diário'].rolling(window=3).mean()
df.dropna(inplace=True)

# Variáveis e normalização
variaveis = ['Preço_Fechamento', 'Máxima', 'Mínima', 'Volume_Negociado', 'Retorno_MM3']
scaler = MinMaxScaler()
dados_normalizados = scaler.fit_transform(df[variaveis])

# Janelas
janela = 45
X, y = [], []
for i in range(janela, len(dados_normalizados)):
    X.append(dados_normalizados[i-janela:i])
    y.append(dados_normalizados[i, 0])

X, y = np.array(X), np.array(y)

# Split
divisao = int(len(X) * 0.8)
X_treino, X_teste = X[:divisao], X[divisao:]
y_treino, y_teste = y[:divisao], y[divisao:]

# Modelo
modelo = Sequential()
modelo.add(LSTM(50, return_sequences=True, input_shape=(janela, X.shape[2])))
modelo.add(Dropout(0.2))
modelo.add(LSTM(50))
modelo.add(Dropout(0.2))
modelo.add(Dense(1))
modelo.compile(optimizer='adam', loss='mean_squared_error')

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
modelo.fit(X_treino, y_treino, validation_data=(X_teste, y_teste), epochs=50, batch_size=32, callbacks=[early_stop])

# Salvar modelo e scaler
save_model(modelo, 'modelo_45dias.h5')
joblib.dump(scaler, 'scaler_45dias.pkl')


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 6s 52ms/step - loss: 0.0307 - val_loss: 0.0057
Epoch 2/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - loss: 0.0026 - val_loss: 0.0018
Epoch 3/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 52ms/step - loss: 0.0020 - val_loss: 0.0013
Epoch 4/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step - loss: 0.0019 - val_loss: 9.6142e-04
Epoch 5/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step - loss: 0.0018 - val_loss: 0.0012
Epoch 6/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - loss: 0.0016 - val_loss: 9.1827e-04
Epoch 7/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 47ms/step - loss: 0.0019 - val_loss: 0.0016
Epoch 8/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 54ms/step - loss: 0.0015 - val_loss: 0.0012
Epoch 9/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 59ms/step - loss: 0.0016 - val_loss: 0.0017
Epoch 10/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 44ms/step - loss: 0.0016 - val_loss: 7.9568e-04
Epoch 11/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step - loss: 0.0014 - val_loss: 0.0011
Epoch 12/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step

['scaler_45dias.pkl']

## Fazer previsão com dados até 2025-05-20 (prever_proximo_valor.py)



In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import joblib
from tensorflow.keras.models import load_model

# Parâmetros
ticker = 'AAPL'
data_final = '2025-05-20'
janela = 45

# Carregar modelo e scaler
modelo = load_model('modelo_45dias.h5')
scaler = joblib.load('scaler_45dias.pkl')

# Coletar dados atualizados
df = yf.download(ticker, start='2018-01-01', end=data_final)
df.columns = df.columns.droplevel(0)
df.columns.name = None
df.columns = ['Preço_Fechamento', 'Máxima', 'Mínima', 'Preço_Abertura', 'Volume_Negociado']
df.reset_index(inplace=True)

# Criar Retorno_MM3
df['Retorno_Diário'] = df['Preço_Fechamento'].pct_change()
df['Retorno_MM3'] = df['Retorno_Diário'].rolling(window=3).mean()
df.dropna(inplace=True)

# Usar as mesmas variáveis
variaveis = ['Preço_Fechamento', 'Máxima', 'Mínima', 'Volume_Negociado', 'Retorno_MM3']
ultimos_dados = df[variaveis].iloc[-janela:]
X_input = scaler.transform(ultimos_dados)
X_input = X_input.reshape(1, janela, len(variaveis))

# Prever
previsao_normalizada = modelo.predict(X_input)
previsao = scaler.inverse_transform(
    np.hstack([previsao_normalizada, np.zeros((1, len(variaveis)-1))])
)[0, 0]

print(f'Previsão do próximo preço de fechamento: {previsao:.2f} USD')

[*********************100%***********************]  1 of 1 completed


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 344ms/step
Previsão do próximo preço de fechamento: 209.12 USD
